In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
#import lightgbm as lgb
import optuna.integration.lightgbm as lgb

from sklearn.model_selection import KFold
from sklearn import preprocessing

In [ ]:
train_df = pd.read_csv('../input/tabular-playground-series-mar-2021/train.csv')
test_df = pd.read_csv('../input/tabular-playground-series-mar-2021/test.csv')

label_encoders = [preprocessing.LabelEncoder() for _ in range(19)]
for i, label_encoder in enumerate(label_encoders):
    label_encoder.fit(pd.concat([train_df.iloc[:,1+i],test_df.iloc[:,1+i]],axis=0))
    train_df.iloc[:,1+i] = label_encoder.transform(train_df.iloc[:,1+i])
    train_df.iloc[:,1+i] = train_df.iloc[:,1+i].astype('category')
    
raw_train_X = train_df.iloc[:,1:-1] #DataFrame
train_y = train_df['target'] #Series
print(raw_train_X.head())
print(train_y.head())

In [ ]:
lgb_train=lgb.Dataset(raw_train_X,train_y)
lgb_train

In [ ]:
params = {'objective':'binary',
          'metric':'auc',
          'verbosity':-1,
         }

tuner = lgb.LightGBMTunerCV(
    params, lgb_train, verbose_eval=100, early_stopping_rounds=100, folds=KFold(n_splits=5), return_cvbooster=True

)

tuner.run()


In [ ]:
print("Best score:", tuner.best_score)
best_params = tuner.best_params
print("Best params:", best_params)
print("  Params: ")
for key, value in best_params.items():
    print("    {}: {}".format(key, value))

In [ ]:
cv_average_model = tuner.get_best_booster()

In [ ]:
test_df = pd.read_csv('../input/tabular-playground-series-mar-2021/test.csv')
for i, label_encoder in enumerate(label_encoders):
    test_df.iloc[:,1+i] = label_encoder.transform(test_df.iloc[:,1+i])
    test_df.iloc[:,1+i] = test_df.iloc[:,1+i].astype('category')
    
raw_test_X = test_df.iloc[:,1:] #DataFrame

y_preds = cv_average_model.predict(raw_test_X, raw_score=True)
y_pred = np.array(y_preds).mean(axis=0)

In [ ]:
y_pred

In [ ]:
submission = pd.DataFrame({'id':test_df['id'],'target':y_pred})
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)